In [1]:
import json
import re
import time
import concurrent.futures
from zhipuai import ZhipuAI

# 初始化 ZhipuAI 客户端
client = ZhipuAI(api_key="ae569c6c31ac4c5c9aa03619d0013a89.ZbCV0v4L6OUpOwOc")

# 定义任务描述模板
task_description_template = """
请对以下作者姓名完成以下两个任务：
1. 将姓名转换为“名的首字母. 姓”的格式。例如，“Yonatan Oren”将转换为“Y. Oren”。
2. 将作者姓名合并为一个字符串，作者之间用逗号连接。如果多于三个作者，用“et.al”表示剩余作者。

作者列表: {authors}

输出格式：
{{
    "转换后的姓名": ["转化后的1", "转化后的2", ...],
    "合并后的字符串": "这里是你合并后的字符串"
}}
一定注意如果多于三个作者，用“et.al”表示剩余作者，合并后的字符串不能有超过3个作者，多余3个后面的一定要用et.al
注意：
1. 请严格按照 JSON 格式输出。
2. 不要包含多余的文本或解释。
3.  我不是让你写代码，直接给我结果json文件
4、一定注意如果多于三个作者，用“et.al”表示剩余作者，合并后的字符串不能有超过3个作者，多余3个后面的一定要用et.al
"""

# 加载论文数据
def load_papers(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        return json.load(f)

# 保存处理后的论文数据
def save_papers(papers, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(papers, f, ensure_ascii=False, indent=4)

# 发送请求并解析结果
def process_paper(paper, idx):
    authors = paper.get('authors', [])
    task_description = task_description_template.format(authors=authors)

    try:
        # 调用大模型 API
        response = client.chat.completions.create(
            model="glm-4-flash",  # 使用 glm-4 模型
            messages=[
                {"role": "system", "content": "论文姓名负责转化和合并功能的机器"},
                {"role": "user", "content": task_description}
            ],
            temperature=0.1
        )

        # 提取回复内容
        response_text = response.choices[0].message.content.strip()

        # 记录大模型返回内容
        with open("model_response_log.txt", "a", encoding="utf-8") as log_file:
            log_file.write(f"Paper-{idx} Response:\n{response_text}\n\n")

        print(f"大模型返回内容 (paper-{idx}): {response_text}")

        # 验证返回内容
        match = re.search(r"({.*?})", response_text, re.DOTALL)
        if match is None:
            raise ValueError("未找到 JSON 格式的内容")

        # 解析 JSON 内容
        json_content = json.loads(match.group(1))
        return {
            "transformed_authors": json_content.get("转换后的姓名", authors),
            "merged_authors": json_content.get("合并后的字符串", "")
        }

    except Exception as e:
        print(f"解析失败: {e} for paper-{idx}")
        
        # 如果解析失败，也记录完整响应
        with open("error_log.txt", "a", encoding="utf-8") as error_file:
            error_file.write(f"Paper-{idx} Error:\n{e}\nResponse:\n{response_text}\n\n")
        
        return {
            "transformed_authors": authors,
            "merged_authors": ""
        }

# 并行处理论文数据
def process_papers_concurrently(papers, max_workers=5):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {executor.submit(process_paper, paper, idx): idx for idx, paper in enumerate(papers, 1)}
        for future in concurrent.futures.as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"任务 paper-{idx} 失败: {e}")
                results.append({
                    "transformed_authors": papers[idx - 1].get('authors', []),
                    "merged_authors": ""
                })
    return results

# 主函数
def main(input_json, output_json, max_workers=5):
    print("加载论文数据...")
    papers = load_papers(input_json)
    print(f"共加载 {len(papers)} 篇论文。")

    print("并行处理论文数据...")
    transformed_results = process_papers_concurrently(papers, max_workers=max_workers)

    print("更新论文数据...")
    for paper, result in zip(papers, transformed_results):
        paper['authors_transformed'] = result.get("transformed_authors", paper.get('authors', []))
        paper['authors_merged'] = result.get("merged_authors", "")

    print("保存处理后的数据...")
    save_papers(papers, output_json)
    print(f"处理完成，已保存到 {output_json}")

if __name__ == "__main__":
    input_json_file = 'paper_metadata_1212_10k.json'  # 输入的JSON文件路径
    output_json_file = 'papers_transformed3.json'  # 输出的JSON文件路径
    max_workers = 8  # 最大并行线程数

    main(input_json_file, output_json_file, max_workers=max_workers)


加载论文数据...
共加载 10526 篇论文。
并行处理论文数据...
大模型返回内容 (paper-4): ```json
{
    "转换后的姓名": ["A. Hemmasian", "A. Barati Farimani"],
    "合并后的字符串": "A. Hemmasian, A. Barati Farimani"
}
```
大模型返回内容 (paper-5): ```json
{
    "转换后的姓名": ["A. Ben Arie", "M. Gorfine"],
    "合并后的字符串": "A. Ben Arie, M. Gorfine"
}
```
大模型返回内容 (paper-8): ```json
{
    "转换后的姓名": ["N. Popp", "J. H. Metzen", "M. Hein"],
    "合并后的字符串": "N. Popp, J. H. Metzen, M. Hein"
}
```
大模型返回内容 (paper-3): ```json
{
    "转换后的姓名": ["S. Chen", "F. Kang", "N. Abhyankar", "M. Jin", "R. Jia"],
    "合并后的字符串": "S. Chen, F. Kang, N. Abhyankar, et al."
}
```
大模型返回内容 (paper-2): ```json
{
    "转换后的姓名": ["M. Verbockhaven", "T. Rudkiewicz", "G. Charpiat", "S. Chevallier"],
    "合并后的字符串": "M. Verbockhaven, T. Rudkiewicz, G. Charpiat, et al."
}
```
大模型返回内容 (paper-6): ```json
{
    "转换后的姓名": ["P. Chao", "P. Blöbaum", "S. K. Patel", "S. Kasiviswanathan"],
    "合并后的字符串": "P. Chao, P. Blöbaum, S. K. Patel, et al."
}
```
大模型返回内容 (paper-10): ```json
{
    "转换后的姓名"